<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/kimi_audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [2]:
!git clone https://github.com/MoonshotAI/Kimi-Audio.git

Cloning into 'Kimi-Audio'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 62 (delta 4), reused 62 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (62/62), 5.71 MiB | 19.83 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [3]:
%cd /content/Kimi-Audio

/content/Kimi-Audio


In [4]:
!git submodule update --init --recursive

Submodule 'kimia_infer/models/tokenizer/glm4' (https://github.com/THUDM/GLM-4-Voice.git) registered for path 'kimia_infer/models/tokenizer/glm4'
Cloning into '/content/Kimi-Audio/kimia_infer/models/tokenizer/glm4'...
Submodule path 'kimia_infer/models/tokenizer/glm4': checked out 'eb00ce9142e8d98b0ed7c57cd47e0d6d5dce9a1a'
Submodule 'third_party/Matcha-TTS' (https://github.com/shivammehta25/Matcha-TTS) registered for path 'kimia_infer/models/tokenizer/glm4/third_party/Matcha-TTS'
Cloning into '/content/Kimi-Audio/kimia_infer/models/tokenizer/glm4/third_party/Matcha-TTS'...
Submodule path 'kimia_infer/models/tokenizer/glm4/third_party/Matcha-TTS': checked out 'dd9105b34bf2be2230f4aa1e4769fb586a3c824e'


In [ ]:
!pip install -q -r requirements.txt

In [ ]:
#!pip install -q torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124

In [6]:
!pip install -q -U flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 114.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
!pip install Ninja


  Using cached ninja-1.11.1.4-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.0 kB)
Using cached ninja-1.11.1.4-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (422 kB)


# download

In [7]:
!huggingface-cli download moonshotai/Kimi-Audio-7B-Instruct --quiet --local-dir /content/models/moonshotai/Kimi-Audio-7B-Instruct

/content/models/moonshotai/Kimi-Audio-7B-Instruct


# inference

In [1]:
import sys
sys.path.insert(0, '/content/Kimi-Audio')

import soundfile as sf
from kimia_infer.api.kimia import KimiAudio

# --- 1. Load Model ---
model_path = "/content/models/moonshotai/Kimi-Audio-7B-Instruct"
model = KimiAudio(model_path=model_path, load_detokenizer=True)

2025-04-26 10:44:02.365 | INFO     | kimia_infer.api.kimia:__init__:16 - Loading kimi-audio main model


Loading checkpoint shards:   0%|          | 0/36 [00:00<?, ?it/s]

2025-04-26 10:44:10.327 | INFO     | kimia_infer.api.prompt_manager:__init__:30 - Looking for resources in /content/models/moonshotai/Kimi-Audio-7B-Instruct
2025-04-26 10:44:10.328 | INFO     | kimia_infer.api.prompt_manager:__init__:31 - Loading whisper model
2025-04-26 10:44:13.993 | INFO     | kimia_infer.api.prompt_manager:__init__:40 - Loading text tokenizer
2025-04-26 10:44:14.253 | INFO     | kimia_infer.api.kimia:__init__:39 - Loading detokenizer
Detected CUDA files, patching ldflags
Emitting ninja build file /content/Kimi-Audio/kimia_infer/models/detokenizer/vocoder/alias_free_activation/cuda/build/build.ninja...
/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:1965: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Building extension module anti_alias_activation_cuda...
Allowing ninja to set a default number of work

Loading '/content/models/moonshotai/Kimi-Audio-7B-Instruct/vocoder/model.pt'
Complete.
using rope base theta = 10000.0, interpolation factor = 1.0
Currently using bfloat16 for PrefixFlowMatchingDetokenizer


In [3]:
import torch
def print_model_params(model: torch.nn.Module, extra_info=""):
    # print the number of parameters in the model
    model_million_params = sum(p.numel() for p in model.parameters()) / 1e6
    print(model)
    print(f"{extra_info} {model_million_params} M parameters")

In [10]:
print_model_params(model.prompt_manager.audio_tokenizer.whisper_model,"Glm4Tokenizer-WhisperVQEncoder")


WhisperVQEncoder(
  (conv1): CausalConv1d(128, 1280, kernel_size=(3,), stride=(1,))
  (conv2): CausalConv1d(1280, 1280, kernel_size=(3,), stride=(2,))
  (embed_positions): Embedding(1500, 1280)
  (layers): ModuleList(
    (0-15): 16 x WhisperVQEncoderLayer(
      (self_attn): WhisperSdpaAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
      )
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (activation_fn): GELUActivation()
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
  )
  (pooling_layer): AvgPool1d(kernel_size=(4,),

In [11]:
print_model_params(model.alm,"qwen2.5-7B-LLM")


MoonshotKimiaForCausalLM(
  (model): MoonshotKimiaModel(
    (embed_tokens): Embedding(168448, 3584, padding_idx=152063)
    (layers): ModuleList(
      (0-27): 28 x MoonshotDecoderLayer(
        (self_attn): MoonshotAttention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps

In [12]:
print_model_params(model.detokenizer.semantic_fm.speech_model,"PrefixStreamingFlowMatchingDetokenizer.semantic_fm_dit")
print_model_params(model.detokenizer.vocoder.vocoder,"PrefixStreamingFlowMatchingDetokenizer.bigvgan")

DiTPrefix(
  (t_embedder): TimestepEmbedder(
    (mlp): Sequential(
      (0): Linear(in_features=256, out_features=2304, bias=True)
      (1): SiLU()
      (2): Linear(in_features=2304, out_features=2304, bias=True)
    )
  )
  (semantic_token_embedding): Embedding(16385, 2304)
  (input_linear): Linear(in_features=80, out_features=2304, bias=True)
  (blocks): ModuleList(
    (0-15): 16 x DiTBlock(
      (norm1): LayerNorm((2304,), eps=1e-06, elementwise_affine=False)
      (attn): Attention(
        (qkv): Linear(in_features=2304, out_features=6912, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=2304, out_features=2304, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (norm2): LayerNorm((2304,), eps=1e-06, elementwise_affine=False)
      (mlp): Mlp(
        (fc1): Linear(in_features=2304, out_features=9216, bias=True)
        (act): GELU(approximate='t

In [ ]:
%cd /content/Kimi-Audio

/content/Kimi-Audio


In [18]:
from IPython.display import Audio
Audio('test_audios/asr_example.wav')

In [15]:
# --- 2. Define Sampling Parameters ---
sampling_params = {
    "audio_temperature": 0.8,
    "audio_top_k": 10,
    "text_temperature": 0.0,
    "text_top_k": 5,
    "audio_repetition_penalty": 1.0,
    "audio_repetition_window_size": 64,
    "text_repetition_penalty": 1.0,
    "text_repetition_window_size": 16,
}

# --- 3. Example 1: Audio-to-Text (ASR) ---
messages_asr = [
    # You can provide context or instructions as text
    {"role": "user", "message_type": "text", "content": "Please transcribe the following audio:"},
    # Provide the audio file path
    {"role": "user", "message_type": "audio", "content": "test_audios/asr_example.wav"}
]

# Generate only text output
_, text_output = model.generate(messages_asr, **sampling_params, output_type="text")
print(">>> ASR Output Text: ", text_output) # Expected output: "这并不是告别，这是一个篇章的结束，也是新篇章的开始。"




Generating tokens:   0%|          | 14/7357 [00:01<12:00, 10.19it/s]


>>> ASR Output Text:  这并不是告别，这是一个篇章的结束，也是新篇章的开始。


In [19]:
from IPython.display import Audio
Audio('test_audios/qa_example.wav')

In [20]:
# --- 4. Example 2: Audio-to-Audio/Text Conversation ---
messages_conversation = [
    # Start conversation with an audio query
    {"role": "user", "message_type": "audio", "content": "test_audios/qa_example.wav"}
]

# Generate both audio and text output
wav_output, text_output = model.generate(messages_conversation, **sampling_params, output_type="both")

# Save the generated audio
output_audio_path = "output_audio.wav"
sf.write(output_audio_path, wav_output.detach().cpu().view(-1).numpy(), 24000) # Assuming 24kHz output
print(f">>> Conversational Output Audio saved to: {output_audio_path}")
print(">>> Conversational Output Text: ", text_output) # Expected output: "A."

print("Kimi-Audio inference examples complete.")

Generating tokens:   5%|▍         | 66/1447 [00:03<01:19, 17.32it/s]
/usr/local/lib/python3.11/dist-packages/torchdyn/numerics/odeint.py:83: UserWarning: Setting tolerances has no effect on fixed-step methods
  warn("Setting tolerances has no effect on fixed-step methods")


>>> Conversational Output Audio saved to: output_audio.wav
>>> Conversational Output Text:  当然可以，这很简单。一二三四五六七八九十。
Kimi-Audio inference examples complete.


In [21]:
from IPython.display import Audio
Audio('output_audio.wav')